In [ ]:
from IPython.display import clear_output,display, update_display
import time
import psycopg2
from psycopg2 import sql
from pybit.unified_trading import HTTP
from datetime import datetime, timezone
from Sdata import bybitkey,bybit_Secret
# 初始化 Bybit 会话
session = HTTP(
    testnet=False,
    api_key=bybitkey,
    api_secret=bybit_Secret
)
# 資料庫連接參數
db_params = {
    "host": "123.241.217.251",
    "database": "mydatabase",
    "user": "myuser",
    "password": "mysecretpassword",
    "port": "5432"
}

# 連接資料庫
def connect_to_db():
    try:
        conn = psycopg2.connect(**db_params)
        return conn
    except (Exception, psycopg2.Error) as error:
        print("連接資料庫時出錯:", error)
        return None

# 執行查詢
def execute_query(conn, query, params=None):
    try:
        with conn.cursor() as cur:
            if params:
                cur.execute(query, params)
            else:
                cur.execute(query)
            conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("執行時出錯:", error)

symbol = "ETHUSDT"
min_qty = 0.01  # 最小下單數量
print('init processing...')
output = display("start trading...", display_id=True)

def close_all_position():
    positions = session.get_positions(category="linear", symbol="ETHUSDT")
    response = session.place_order(
    category="linear",  # 或 "inverse"，根據合約類型設定
    symbol="ETHUSDT",
    side="Sell",  # 平掉多單的方向為 "Sell"
    orderType="Market",
    qty=positions['result']['list'][0]['size'],
    reduceOnly=True,  # 設定為 True 確保這是一個平倉訂單
    timeInForce="GTC",  # Good Till Cancel
    positionIdx=0, #平掉多單是1
    )
    return response

def print_with_clear(message):
    clear_output(wait=True)
    print(message)


def place_order(side, qty):
    try:
        order = session.place_order(
            category="linear",
            symbol=symbol,
            side=side,
            orderType="Market",
            qty=qty
        )
        return order
    except Exception as e:
        print(f"order fail: {e}")
        return None

def get_position():
    try:
        position = session.get_positions(category="linear", symbol=symbol)
        return position['result']['list'][0]
    except Exception as e:
        print(f"获取持仓信息失败: {e}")
        return None

def calculate_roi(position):
    entry_price = float(position['avgPrice'])
    current_price = float(position['markPrice'])
    return (current_price - entry_price) / entry_price * 100

def trading_strategy():
    while True:
        position = get_position()
        if position is None:
            continue

        size = float(position['size'])
        
        if size == 0:
            # if not order，start treading (small_order)
            place_order("Buy", min_qty)
            input_db_trading_data()
        else:
            roi = calculate_roi(position) * 100
            # print(f'ROI: {roi} %')
            # print_with_clear(f'ROI: {roi} %')
            update_display(f'ROI: {roi} % time=>{datetime.now()}', display_id=output.display_id)
            time.sleep(1)
            # ROI == +20% 平倉
            # if roi >= 20:
            #     ans = close_all_position()
            #     print('平倉',ans)
            # elif roi <= -50:
            #     print('Order puls Buy')
            #     # ROI 达到 -50%，加仓
            #     place_order("Buy", min_qty)
            
            if roi <= -77:
                print('Order puls Buy',datetime.now())
                place_order("Buy", min_qty)
                input_db_trading_data()
                

            
def input_db_trading_data():
    conn = connect_to_db()
    positions = session.get_positions(category="linear", symbol="ETHUSDT")

    symbol= positions['result']['list'][0]['symbol']
    PositionIM= positions['result']['list'][0]['positionIM'] 
    Leverage= positions['result']['list'][0]['leverage']
    markPrice= positions['result']['list'][0]['markPrice'] 
    AvgPrice= positions['result']['list'][0]['avgPrice'] 
    Size= positions['result']['list'][0]['size']
    Side= positions['result']['list'][0]['side']
    CurRealisedPnl= positions['result']['list'][0]['curRealisedPnl'] 
    PositionValue= positions['result']['list'][0]['positionValue'] 
    createdTime= datetime.fromtimestamp(int(positions['result']['list'][0]['updatedTime']) / 1000, timezone.utc).astimezone().strftime('%Y-%m-%d %H:%M:%S')
    Time= datetime.fromtimestamp(int(positions['time']) / 1000, timezone.utc).astimezone().strftime('%Y-%m-%d %H:%M:%S')
    if conn:
        insert_query = sql.SQL("""
        INSERT INTO sptd.tdata 
        (symbol, asset, leverage, buy_price, sell_price, quantity, transaction_type, CurRealisedPnl, PositionValue, create_transaction_time, transaction_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """)
        
        params = (
            symbol, PositionIM, Leverage, markPrice, AvgPrice, Size, Side, 
            CurRealisedPnl, PositionValue, createdTime, Time
        )
        
        execute_query(conn, insert_query, params)

        conn.close()
        

if __name__ == "__main__":
    trading_strategy()

init processing...


'ROI: 201.45544445596533 % time=>2024-10-16 19:33:16.333834'